In [ ]:
# !pip install catboost
!pip install pandas
!pip install numpy
!pip install pymorphy2
# !pip install scikit-learn
# !pip install matplotlib
# !pip install seaborn
# !pip install scipy

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from datetime import datetime

In [9]:
from google.colab import drive
drive.mount('/content/drive')
# file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/Данные_ПиОА_2023_1семестр.xlsx"
file_name = "/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/data/трек HARD.xlsx"

xl2 = None
if file_name.endswith('.xlsx'):
    xl2 = pd.ExcelFile(
        file_name,
        engine='openpyxl'
    )
elif file_name.endswith('.xls'):
    xl2 = pd.ExcelFile(
        file_name,
        engine='xlrd'
    )
print(xl2.sheet_names)

xls_array = [xl2.parse(sheet) for sheet in xl2.sheet_names]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Информация по студентам', 'Информация по студентам_', 'Информация по студентам__', 'Преподаватели Л, П']


In [4]:
directory_name = "Учет контрольных"


In [ ]:
import tkinter.filedialog as tkFileDialog
directory_name = "Учет контрольных"

def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')

In [ ]:
file_name = open_file()
xl = None
if file_name.endswith('.xlsx'):
    xl = pd.ExcelFile(
        file_name,
        engine='openpyxl'
    )
elif file_name.endswith('.xls'):
    xl = pd.ExcelFile(
        file_name,
        engine='xlrd'
    )
print(xl.sheet_names)

xls_array = [xl.parse(sheet) for sheet in xl.sheet_names]

# for sheet in xls_array:
#     print(sheet.columns)

['Информация по студентам', 'Информация по студентам_', 'Информация по студентам__', 'Преподаватели Л, П']


In [5]:
import re
import pymorphy2
genders = {"masc": 1, "femn": 0}
morph = pymorphy2.MorphAnalyzer()

def GetGender(full_name):
    name = re.search(r"(?:.* )([^ ]*)(?:[ .]*)", full_name).group(1)
    gender = morph.parse(name)[0].tag.gender
    return genders[gender] if gender in genders.keys() else 1.0

In [10]:
# расчет для контрольных
import os

if not os.path.exists(directory_name):
    os.makedirs(directory_name)

temp_row = ""
for sheet in xls_array:
    for name, group in sheet.groupby(["Название РМУП"]):
        if "Предмет контроля" not in group.columns:
            continue
        name_column_test = ""
        if "Контрольная работа" in set(list(group["Предмет контроля"])):
            name_column_test = "Контрольная работа"
        else:
            name_column_test = "Работа на учебной встрече"
        results = pd.DataFrame(columns=["ФИО", "Пол", "Команда", "Баллы", "Не сдал(-а)"])

        name_discipline = group.head(1)["Название РМУП"].values[0]
        try:
            if (group["Название встречи"].str.contains('Контрольная ').sum()==0 and name_column_test!="Контрольная работа"):
                continue
            print(name_discipline, end=":")
            print(name_column_test, set(list(group["Предмет контроля"])))

            max_scores_before_test = {}

            percent = 0
            count_presence = 0
            summ_presence = 0
            count_goals = 0
            startTime = datetime.now()
            # номер контрольной
            test_number = 1
            # # количество действий с кр (баллы и результаты до)
            # count_test_item = 0
            for index, row in group.iterrows ():

                if (index%(len(group)//10)==0):
                    print(f"{percent}%. Записей в итоге: {len(results)}")
                    percent+=10

                if temp_row == "" or temp_row != row["ФИО студента"]:
                    test_number = 1
                    count_presence = summ_presence = count_goals = 0.0
                    temp_row = row["ФИО студента"]
                    results.loc[-1] = [temp_row, GetGender(temp_row), row["Команда"], row["Итог ТУ"], 1 if row["Итоговая оценка"]=="неудовл." or row["Итоговая оценка"]=="" else 0] + [""]*(len(results.columns)-5)
                    # results.loc[-1] = [temp_row, GetGender(temp_row), row["Команда"], row["Итог ТУ"], GetInfoLose(row["Итог ТУ"])] + [""]*(len(results.columns)-5)
                    results.index = results.index + 1  # shifting index

                if ("Контрольная " in row["Название встречи"] or "Контрольная работа" == row["Предмет контроля"]):
                    meeting_name = f"Контрольная работа {test_number}"

                    if (f"Контрольная работа {test_number}" not in list(results.columns)):
                        if (f"Посещение до {meeting_name}" not in list(results.columns)):
                            results[f"Посещение до {meeting_name}"] = 0.0
                            results[f"Баллы до {meeting_name}"] = 0.0
                            results[f"Процент баллов до {meeting_name}"] = 0.0
                        results[meeting_name] = 0.0
                    if (row["Предмет контроля"] == name_column_test):

                        results.loc[((results['ФИО'] == temp_row)), f"Контрольная работа {test_number}"] = row["Оценка за предметы контроля"] if row["Оценка за предметы контроля"]!="" else 0 # row["Название встречи"]

                    # if (row["Предмет контроля"] != name_column_test or name_column_test == "Контрольная работа"):

                        if meeting_name not in max_scores_before_test:
                            max_scores_before_test[meeting_name] = count_goals
                        elif max_scores_before_test[meeting_name] < count_goals:
                            max_scores_before_test[meeting_name] = count_goals

                        if count_presence == 0: #or results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"].values[0]!=""
                            continue
                        results.loc[((results['ФИО'] == temp_row)), f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                        results.loc[((results['ФИО'] == temp_row)), f"Баллы до {meeting_name}"] = count_goals*1.0
                        results.loc[((results['ФИО'] == temp_row)), f"Процент баллов до {meeting_name}"] = count_goals*1.0
                        count_presence = summ_presence = count_goals = 0.0
                        test_number+=1


                elif (row["Предмет контроля"] != "Посещение"):
                    if not pd.isna(row["Оценка за предметы контроля"]):
                        count_goals += float(row["Оценка за предметы контроля"])
                else:
                    count_presence += 1
                    if row["Оценка за предметы контроля"]=="П":
                        summ_presence += 1
            results = results.fillna(0)
            results = results.replace("", 0)
            results = results.drop(results.columns[-4:], axis= 1)
            # results = DeleteAnomalies(results)

            del max_scores_before_test[f"Контрольная работа {test_number}"]
            for meeting_name, max_score in max_scores_before_test.items():
                if max_score>0:
                    convert_dict = {f"Посещение до {meeting_name}": float,
                                        f"Баллы до {meeting_name}": float,
                                        f"Процент баллов до {meeting_name}": float,
                                        meeting_name: float,
                                        "Баллы": float
                                    }
                    results = results.astype(convert_dict)
                    results[f"Процент баллов до {meeting_name}"] = results[f"Процент баллов до {meeting_name}"].div(max_score*1.0).round(2)

            print(f"Время выполнения: {datetime.now()-startTime}")
            print("Отдельное сохранение...")
            # results.to_csv(f"{directory_name}\{name_discipline}.csv", index=False, sep=';', encoding='cp1251')
            results.to_csv(f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}/{name_discipline} новый.csv", index=False, sep=';', encoding='cp1251')
            print("Сохранено")
        except Exception as e:
            print(e)



Алгебра:Работа на учебной встрече {'Работа на учебной встрече', 'Посещение'}
0%. Записей в итоге: 94
10%. Записей в итоге: 189
20%. Записей в итоге: 285
30%. Записей в итоге: 380
40%. Записей в итоге: 475
50%. Записей в итоге: 570
60%. Записей в итоге: 666
70%. Записей в итоге: 760
80%. Записей в итоге: 856
90%. Записей в итоге: 951
Время выполнения: 0:00:14.040633
Отдельное сохранение...
Сохранено
Программирование и основы алгоритмизации 1:Контрольная работа {'Работа на учебной встрече', 'Посещение', 'Контрольная работа'}
0%. Записей в итоге: 84
10%. Записей в итоге: 180
20%. Записей в итоге: 276
30%. Записей в итоге: 372
40%. Записей в итоге: 468
50%. Записей в итоге: 564
60%. Записей в итоге: 661
70%. Записей в итоге: 756
80%. Записей в итоге: 852
90%. Записей в итоге: 948
Время выполнения: 0:00:13.364170
Отдельное сохранение...
Сохранено
Математический анализ:Контрольная работа {'Практическое задание', 'Работа на учебной встрече', 'Письменный ответ', 'Посещение', 'Контрольная работ

In [ ]:
# файл ПиОА.xlsx
file_name = open_file()
xl2 = None
if file_name.endswith('.xlsx'):
    xl2 = pd.ExcelFile(
        file_name,
        engine='openpyxl'
    )
elif file_name.endswith('.xls'):
    xl2 = pd.ExcelFile(
        file_name,
        engine='xlrd'
    )
print(xl2.sheet_names)

xls_array2 = [xl2.parse(sheet) for sheet in xl2.sheet_names]


# for sheet in xls_array:
#     print(sheet.columns)

In [ ]:
for sheet in xls_array:
    # print(sheet.loc[0, :].values.flatten().tolist())
    print(sheet.columns)
    sheet.columns = sheet.loc[0, :].values.flatten().tolist()
    sheet.drop(index=[0], inplace=True)
    sheet = sheet.reset_index(drop=True)
    # sheet.dropna(subset=['Присутствие'], inplace=True)
    print(sheet.columns)

In [8]:
list_result_mark = ["удовл.", "хор.", "отл."]

def GetInfoLose(row):
    if float(row["Итог текущ."])>=61:
        return 0
    return 1 if row["Итоговая оценка"] not in list_result_mark else 0

data_results_presence2 = pd.DataFrame(columns=["Команды", "Направление", "Пол", "Баллы", "Не сдал(-а)"])
print(data_results_presence2.columns)
for sheet in xls_array:
    print(f"Записей: {len(sheet)}")
    startTime = datetime.now()
    percent = 0

    # try:
    max_scores_before_test = {}

    for index, row in sheet.iterrows ():
        if (index%(len(sheet)//10)==0):
            print(f"{percent}%. Записей в итоге: {len(data_results_presence2)}")
            percent+=10
        data_results_presence2.loc[-1] = [row["Учебные команды"], row["Направление подготовки"], GetGender(row["Обучающийся"]), row["Итог текущ."], GetInfoLose(row)] + [""]*(len(data_results_presence2.columns)-5)
        data_results_presence2.index = data_results_presence2.index + 1

        count_presence = summ_presence = count_kontr = 0
        count_goals = 0
        count_kontr = 0
        for i in range(5, len(sheet.columns.to_list())):
            if sheet.columns.to_list()[i]!="Контрольная работа":
                if sheet.columns[i]=="Присутствие":
                    count_presence+=1
                    if (row[i]=="П"):
                        summ_presence+=1
                elif sheet.columns[i]=="Работа на учебной встрече":
                    if row[i]!="" and row[i]:
                        goal = (float(row[i]) if "." in str(row[i]) else float(str(row[i]).replace(",", ".")))
                        if not pd.isna(goal):
                            count_goals += goal
            else:
                count_kontr+=1
                meeting_name = f"Контрольная работа {count_kontr}"

                if meeting_name not in max_scores_before_test:
                    max_scores_before_test[meeting_name] = count_goals

                elif max_scores_before_test[meeting_name] < count_goals:
                    max_scores_before_test[meeting_name] = count_goals

                if (f"Посещение до {meeting_name}" not in list(data_results_presence2.columns)):
                    data_results_presence2[f"Посещение до {meeting_name}"] = 0
                    data_results_presence2[f"Баллы до {meeting_name}"] = 0
                    data_results_presence2[f"Процент баллов до {meeting_name}"] = 0
                    data_results_presence2[meeting_name] = 0
                    convert_dict = {f"Посещение до {meeting_name}": float,
                                    f"Баллы до {meeting_name}": float,
                                    f"Процент баллов до {meeting_name}": float,
                                    meeting_name: float,
                                    "Баллы": float
                                    }
                    data_results_presence2 = data_results_presence2.astype(convert_dict)
                # if count_presence == 0:
                #     count_presence = summ_presence = count_goals = 0
                #     continue
                data_results_presence2.loc[data_results_presence2.index[-1], f"Посещение до {meeting_name}"] = round(summ_presence / float(count_presence), 2)
                data_results_presence2.loc[data_results_presence2.index[-1], f"Баллы до {meeting_name}"] = count_goals
                data_results_presence2.loc[data_results_presence2.index[-1], f"Процент баллов до {meeting_name}"] = count_goals
                data_results_presence2.loc[data_results_presence2.index[-1], meeting_name]  = float(row[i])

                count_presence = summ_presence = count_goals = 0

    data_results_presence2 = data_results_presence2.fillna(0)
    for meeting_name, max_score in max_scores_before_test.items():
        data_results_presence2[f"Процент баллов до {meeting_name}"] = data_results_presence2[f"Процент баллов до {meeting_name}"].div(max_score*1.0).round(2)
    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    print(f"Время выполнения: {datetime.now()-startTime}")
    print("Отдельное сохранение...")
    data_results_presence2.to_csv(f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/{directory_name}/Программирование и основы алгоритмизации тест новый.csv", index=False, sep=';', encoding='cp1251')
    print("Сохранено")
    # except Exception as e:
    #     print("Ошибка:", e)

Index(['Команды', 'Направление', 'Пол', 'Баллы', 'Не сдал(-а)'], dtype='object')
Записей: 961
0%. Записей в итоге: 95
10%. Записей в итоге: 191
20%. Записей в итоге: 287
30%. Записей в итоге: 383
40%. Записей в итоге: 479
50%. Записей в итоге: 575
60%. Записей в итоге: 671
70%. Записей в итоге: 767
80%. Записей в итоге: 863
90%. Записей в итоге: 959
Время выполнения: 0:00:05.936804
Отдельное сохранение...
Время выполнения: 0:00:05.937998
Отдельное сохранение...
Сохранено
